# OCR فارسی - تبدیل PDF به متن

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MohammadHNdev/Persian_OCR_Simple/blob/main/Persian_OCR_Optimized.ipynb)

یه ابزار ساده برای خوندن متن فارسی از روی PDF ها. سریعه و کیفیتش خوبه.

**نحوه استفاده:**
1. همه خونه‌ها رو اجرا کن
2. فایل PDF ات رو آپلود کن
3. یکم منتظر بمون
4. فایل متنی رو دانلود کن

In [ ]:
# نصب پایتون ها
print("باید این چیزها رو نصب کنم...")
# اینو از نت کپی کردم
!pip install pytesseract pdf2image Pillow tqdm --quiet
!sudo apt update --quiet
!sudo apt install -y tesseract-ocr tesseract-ocr-fas poppler-utils --quiet
print("Ok نصب شد")

In [ ]:
# import ها
import pytesseract
from PIL import Image
from PIL import ImageEnhance  # جدا import کردم چون آموزش اینطور گفت
from pdf2image import convert_from_path
import os
import time
import subprocess
import shutil
import gc  # garbage collector
from tqdm.notebook import tqdm
import concurrent.futures
from google.colab import files

x = 1  # برای تست
if x == 1:
    print("همه چیز import شد")

In [ ]:
# این تابع رو برای خوندن یک صفحه نوشتم
def do_ocr_on_one_page(image_file_path, page_number):
    def improve_image_quality(img):
        # این قسمت رو از استک اورفلو کپی کردم
        gray = img.convert('L')
        enhancer = ImageEnhance.Contrast(gray)
        enhanced = enhancer.enhance(2.5)
        return enhanced
    
    def make_sharp(img):
        # این هم برای تیز کردن
        sharpener = ImageEnhance.Sharpness(img)
        sharp_img = sharpener.enhance(1.3)
        return sharp_img
    
    try:
        # عکس رو باز می‌کنم
        image = Image.open(image_file_path)
        
        # بهتر می‌کنم
        better_image = improve_image_quality(image)
        final_image = make_sharp(better_image)
        
        # حالا tesseract رو اجرا می‌کنم
        extracted_text = pytesseract.image_to_string(final_image, lang='fas', config='--psm 3 --oem 3')
        
        # حافظه رو پاک می‌کنم
        image.close()
        better_image = None
        final_image = None
        del image
        del better_image  
        del final_image
        
        # فایل موقت رو پاک می‌کنم
        try:
            os.remove(image_file_path)
        except:
            pass
        
        # نتیجه
        result = {}
        result['page_num'] = page_number
        result['text_content'] = extracted_text.strip()
        result['character_count'] = len(extracted_text.strip())
        result['is_successful'] = True
        
        return result
        
    except Exception as error:
        # اگه خطا خورد
        error_result = {}
        error_result['page_num'] = page_number
        error_result['text_content'] = f"خطا در صفحه {page_number}: {str(error)}"
        error_result['character_count'] = 0
        error_result['is_successful'] = False
        return error_result

# تابع برای پیدا کردن تعداد صفحات PDF
def find_pdf_pages(pdf_file_path):
    page_count = 0
    
    # روش اول - استفاده از pdfinfo
    try:
        result = subprocess.run(['pdfinfo', pdf_file_path], capture_output=True, text=True)
        output_lines = result.stdout.split('\n')
        for line in output_lines:
            if 'Pages:' in line:
                page_count = int(line.split(':')[1].strip())
                break
        if page_count > 0:
            return page_count
    except:
        print("pdfinfo کار نکرد")
    
    # روش دوم - تست کردن
    try:
        test_images = convert_from_path(pdf_file_path, dpi=72, first_page=1, last_page=5)
        estimated_pages = len(test_images) * 20
        return estimated_pages
    except:
        print("روش دوم هم کار نکرد")
    
    # اگه هیچی کار نکرد
    return 100

# تست
test_var = "ok"
print("توابع کمکی نوشته شدن - " + test_var)

In [ ]:
# تبدیل PDF به عکس
def convert_pdf_to_image_files(input_pdf_path, batch_size_for_processing=8, image_dpi_quality=350):
    def create_temp_directory():
        # پوشه موقت می‌سازم
        temp_directory_path = '/tmp/temp_ocr_images'
        if os.path.exists(temp_directory_path):
            shutil.rmtree(temp_directory_path)
        os.makedirs(temp_directory_path)  
        return temp_directory_path
    
    def save_images_to_disk(images_list, start_page_num, temp_dir):
        # این تابع عکس‌ها رو ذخیره می‌کنه
        saved_paths = []
        counter = 0
        for single_image in images_list:
            page_number = start_page_num + counter
            file_name = f"page_{page_number:04d}.jpg"
            full_path = os.path.join(temp_dir, file_name)
            single_image.save(full_path, 'JPEG', quality=95, optimize=True)
            saved_paths.append((full_path, page_number))
            counter = counter + 1
        return saved_paths
    
    def cleanup_memory(images_to_cleanup):
        # حافظه رو پاک می‌کنم
        for img in images_to_cleanup:
            del img
        del images_to_cleanup
        gc.collect()
    
    print("شروع تبدیل PDF...")
    
    # تعداد صفحات رو پیدا می‌کنم
    total_pages = find_pdf_pages(input_pdf_path)
    print(f"کل صفحات: {total_pages}")
    
    # پوشه موقت
    temp_folder = create_temp_directory()
    
    # لیست همه عکس‌ها
    all_image_paths = []
    
    # حلقه اصلی برای پردازش دسته‌ای
    current_start_page = 1
    while current_start_page <= total_pages:
        current_end_page = current_start_page + batch_size_for_processing - 1
        if current_end_page > total_pages:
            current_end_page = total_pages
        
        print(f"در حال پردازش صفحات {current_start_page} تا {current_end_page}")
        
        try:
            # تبدیل PDF به عکس برای این دسته
            converted_images = convert_from_path(
                input_pdf_path,
                dpi=image_dpi_quality,
                first_page=current_start_page,
                last_page=current_end_page,
                fmt='jpeg',
                thread_count=2
            )
            
            # ذخیره عکس‌ها
            batch_image_paths = save_images_to_disk(converted_images, current_start_page, temp_folder)
            all_image_paths.extend(batch_image_paths)
            
            # پاک کردن حافظه
            cleanup_memory(converted_images)
            
        except Exception as conversion_error:
            if current_start_page <= total_pages:
                print(f"مشکل در دسته {current_start_page}-{current_end_page}: {str(conversion_error)}")
        
        # به دسته بعدی می‌رم
        current_start_page = current_start_page + batch_size_for_processing
    
    print(f"در مجموع {len(all_image_paths)} عکس ساخته شد")
    return all_image_paths

# تست کردن
temp_result = "تبدیل‌کننده PDF"
print(temp_result + " آماده شد")

In [ ]:
# تابع اصلی OCR
def main_ocr_processing_function(pdf_file_path, number_of_workers=3, dpi_for_images=350):
    def calculate_processing_statistics(results_from_ocr):
        # محاسبه آمار
        successful_results = []
        total_character_count = 0
        for result in results_from_ocr:
            if result['is_successful'] and result['character_count'] > 5:
                successful_results.append(result)
                total_character_count = total_character_count + result['character_count']
        return successful_results, total_character_count
    
    def generate_output_filename():
        # نام فایل خروجی
        import time
        current_timestamp = int(time.time())
        filename = f"persian_ocr_result_{current_timestamp}.txt"
        return filename
    
    def write_results_to_file(all_results, output_filename, source_pdf_name, total_chars, successful_count, total_count):
        # نوشتن در فایل
        with open(output_filename, 'w', encoding='utf-8') as file_handle:
            # هدر فایل
            file_handle.write("# نتایج OCR فارسی\n")
            file_handle.write(f"# فایل مبدا: {source_pdf_name}\n")
            file_handle.write(f"# تاریخ: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
            file_handle.write(f"# صفحات موفق: {successful_count} از {total_count}\n")
            file_handle.write(f"# تعداد کاراکتر: {total_chars:,}\n")
            file_handle.write("\n" + "="*70 + "\n\n")
            
            # محتوای هر صفحه
            for single_result in all_results:
                file_handle.write(f"\n--- صفحه {single_result['page_num']} ---\n")
                if single_result['is_successful']:
                    file_handle.write(f"({single_result['character_count']} کاراکتر)\n")
                file_handle.write(single_result['text_content'])
                file_handle.write("\n\n")
    
    def cleanup_temp_files():
        # پاک کردن فایل‌های موقت
        try:
            shutil.rmtree('/tmp/temp_ocr_images')
            print("فایل‌های موقت پاک شدند")
        except:
            print("مشکل در پاک کردن فایل‌های موقت")
    
    print(f"شروع پردازش: {os.path.basename(pdf_file_path)}")
    start_processing_time = time.time()
    
    # گرفتن لیست عکس‌ها
    list_of_image_paths = convert_pdf_to_image_files(pdf_file_path, image_dpi_quality=dpi_for_images)
    
    if len(list_of_image_paths) == 0:
        print("هیچ عکسی تولید نشد!")
        return None
    
    print(f"شروع OCR با {number_of_workers} worker...")
    
    # لیست نتایج
    all_ocr_results = []
    
    # پردازش موازی
    with concurrent.futures.ThreadPoolExecutor(max_workers=number_of_workers) as executor:
        # سایز دسته
        batch_processing_size = 12
        
        # شمارنده دسته
        batch_counter = 0
        
        while batch_counter < len(list_of_image_paths):
            # محاسبه انتهای دسته
            batch_end_index = batch_counter + batch_processing_size
            if batch_end_index > len(list_of_image_paths):
                batch_end_index = len(list_of_image_paths)
            
            # دسته جاری
            current_processing_batch = list_of_image_paths[batch_counter:batch_end_index]
            
            # شماره دسته برای نمایش
            display_batch_number = (batch_counter // batch_processing_size) + 1
            total_batches = ((len(list_of_image_paths) - 1) // batch_processing_size) + 1
            print(f"دسته {display_batch_number} از {total_batches}")
            
            # ارسال کارها
            submitted_jobs = {}
            for image_path_info in current_processing_batch:
                image_path = image_path_info[0]
                page_num = image_path_info[1]
                job = executor.submit(do_ocr_on_one_page, image_path, page_num)
                submitted_jobs[job] = page_num
            
            # جمع‌آوری نتایج
            for completed_job in tqdm(concurrent.futures.as_completed(submitted_jobs), 
                                    total=len(submitted_jobs), 
                                    desc=f"دسته {display_batch_number}"):
                try:
                    job_result = completed_job.result(timeout=30)
                    all_ocr_results.append(job_result)
                except Exception as job_error:
                    failed_page_number = submitted_jobs[completed_job]
                    error_result = {
                        'page_num': failed_page_number,
                        'text_content': f"خطا: {str(job_error)}",
                        'character_count': 0,
                        'is_successful': False
                    }
                    all_ocr_results.append(error_result)
            
            # به دسته بعدی
            batch_counter = batch_end_index
            gc.collect()
    
    # مرتب کردن نتایج بر اساس شماره صفحه
    all_ocr_results.sort(key=lambda x: x['page_num'])
    
    # محاسبه آمار
    successful_pages, total_characters = calculate_processing_statistics(all_ocr_results)
    
    print("تولید فایل خروجی...")
    
    # نام فایل خروجی
    output_file_name = generate_output_filename()
    
    # نوشتن فایل
    write_results_to_file(
        all_ocr_results, 
        output_file_name, 
        os.path.basename(pdf_file_path),
        total_characters,
        len(successful_pages),
        len(all_ocr_results)
    )
    
    # محاسبه زمان
    end_processing_time = time.time()
    processing_duration = end_processing_time - start_processing_time
    
    # نمایش نتایج
    print("\nپردازش تمام شد!")
    success_rate = len(successful_pages) / len(all_ocr_results)
    print(f"موفقیت: {len(successful_pages)}/{len(all_ocr_results)} ({success_rate*100:.1f}%)")
    print(f"کل کاراکترها: {total_characters:,}")
    print(f"زمان: {processing_duration:.1f} ثانیه ({processing_duration/60:.1f} دقیقه)")
    if processing_duration > 0:
        print(f"سرعت: {total_characters/processing_duration:.0f} کاراکتر/ثانیه")
    print(f"فایل خروجی: {output_file_name}")
    
    # ارزیابی کیفیت
    if success_rate >= 0.9:
        quality_description = "عالی!"
    elif success_rate >= 0.7:
        quality_description = "خوب"
    else:
        quality_description = "متوسط"
    
    print(f"کیفیت: {quality_description}")
    
    # نمایش نمونه
    if len(successful_pages) > 0:
        best_page = successful_pages[0]
        for page in successful_pages:
            if page['character_count'] > best_page['character_count']:
                best_page = page
        
        sample_text = best_page['text_content'][:200]
        print(f"\nنمونه از صفحه {best_page['page_num']}:")
        print("-" * 40)
        print(sample_text)
        if len(best_page['text_content']) > 200:
            print("...")
    
    # دانلود فایل
    try:
        files.download(output_file_name)
        print(f"\nفایل {output_file_name} در حال دانلود...")
    except Exception as download_error:
        print(f"\nخطا در دانلود: {str(download_error)}")
        print("فایل در پنل فایل‌ها موجود است")
    
    # پاک کردن فایل‌های موقت
    cleanup_temp_files()
    
    # برگرداندن نتیجه
    final_result = {
        'processing_completed': True,
        'output_filename': output_file_name,
        'processing_statistics': {
            'total_pages_processed': len(all_ocr_results),
            'successful_pages_count': len(successful_pages),
            'total_characters_extracted': total_characters,
            'processing_time_seconds': processing_duration,
            'quality_assessment': quality_description
        }
    }
    
    return final_result

# متغیر تست
test_message = "تابع اصلی OCR"
print(test_message + " نوشته شد!")

In [ ]:
# آپلود فایل PDF
print("فایل PDF رو اینجا بذار:")
print("(اگه کیفیت اسکن خوب باشه، نتیجه بهتر میشه)")

# آپلود کردن فایل
uploaded_files = files.upload()

# پیدا کردن فایل PDF
user_pdf_file = None
if uploaded_files:
    # چک کردن همه فایل‌ها
    for uploaded_filename in uploaded_files.keys():
        if uploaded_filename.lower().endswith('.pdf'):
            user_pdf_file = uploaded_filename
            break
    
    # اگه PDF پیدا شد
    if user_pdf_file:
        # محاسبه سایز فایل
        file_size_in_megabytes = len(uploaded_files[user_pdf_file]) / (1024*1024)
        print(f"فایل گرفته شد: {user_pdf_file} ({file_size_in_megabytes:.1f} مگابایت)")
        
        # تخمین زمان - این عدد رو از تجربه گذاشتم
        estimated_processing_time = file_size_in_megabytes * 0.2
        print(f"تخمین زمان: {estimated_processing_time:.1f} دقیقه")
        
        # اگه فایل بزرگ باشه
        if file_size_in_megabytes > 30:
            print("فایل کمی بزرگه! شاید بیشتر طول بکشه")
    else:
        print("فایل PDF پیدا نکردم!")
else:
    print("هیچ فایلی آپلود نشد")

In [ ]:
# اجرای OCR
if 'user_pdf_file' in locals() and user_pdf_file:
    print(f"شروع پردازش: {user_pdf_file}")
    print("کمی صبر کن...")
    
    # تنظیمات بر اساس سایز فایل
    pdf_size_mb = len(uploaded_files[user_pdf_file]) / (1024*1024)
    
    # این تنظیمات رو خودم آزمایش کردم
    if pdf_size_mb < 5:
        image_quality_dpi = 400
        worker_threads_count = 3
    elif pdf_size_mb < 20:
        image_quality_dpi = 350  
        worker_threads_count = 3
    else:
        image_quality_dpi = 300  # فایل بزرگ - محتاط‌انه
        worker_threads_count = 2
    
    print(f"تنظیمات: DPI={image_quality_dpi}, Workers={worker_threads_count}")
    
    # شروع پردازش اصلی
    final_processing_result = main_ocr_processing_function(
        user_pdf_file, 
        number_of_workers=worker_threads_count, 
        dpi_for_images=image_quality_dpi
    )
    
    # بررسی نتیجه
    if final_processing_result and final_processing_result['processing_completed']:
        processing_stats = final_processing_result['processing_statistics']
        print(f"\nکامل شد! {processing_stats['quality_assessment']}")
        print(f"کاراکترها: {processing_stats['total_characters_extracted']:,} در {processing_stats['processing_time_seconds']:.1f} ثانیه")
        print(f"فایل نهایی: {final_processing_result['output_filename']}")
    else:
        print("متاسفانه مشکلی پیش آمد :(")
else:
    print("ابتدا فایل PDF را آپلود کنید!")

## کار تمام!

اگه مشکلی بود، بهم بگو تا درستش کنم.

**نکات مهم:**
- کیفیت اسکن خیلی تاثیر داره
- فایل‌های کوچیک‌تر سریع‌تر پردازش میشن  
- اگه کرش کرد، restart کن و دوباره امتحان کن

**برای استفاده مجدد:**
1. Runtime -> Restart and run all کن
2. فایل جدید آپلود کن
3. منتظر بمون

اگه کارت راه افتاد، با دوستات هم share کن!